In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyvista as pv
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path

In [3]:
p  = !pwd
root=Path(p[0])

In [27]:
ROOT = root.parent
marv_obj = ROOT / "meshes/marv_lid.obj"
test_mesh = pv.read(marv_obj)

In [28]:
test_mesh.plot()

Widget(value='<iframe src="http://localhost:63615/index.html?ui=P_0x368beb310_10&reconnect=auto" class="pyvist…

In [29]:
import yaml
with open(root.parent / f"robots/marv.yaml", "r") as file:
    robot_params = yaml.safe_load(file)

In [30]:
import torch
driving_part_bboxes = [torch.tensor(bbox) for bbox in robot_params["driving_part_bboxes"]]
body_bbox = torch.tensor(robot_params["body_bbox"])

In [31]:
robot_points = torch.tensor(pv.read(str(marv_obj)).points)

In [32]:
assert test_mesh.is_all_triangles

In [33]:
import pyacvd


def extract_flipper_points_from_mesh(mesh: pv.PolyData, n_points: int = 100, **clus_opts) -> torch.Tensor:
    delaunay = mesh.delaunay_3d()
    surf = delaunay.extract_surface()
    clus: pyacvd.Clustering = pyacvd.Clustering(surf)
    clus.cluster(n_points, **clus_opts)
    return torch.tensor(clus.cluster_centroid)

In [34]:
from flipper_training.utils.geometry import points_in_oriented_box
driving_meshes = []
driving_points = []
faces = test_mesh.faces.reshape(-1, 4)[:, 1:]
for p in driving_part_bboxes:
    mask = points_in_oriented_box(robot_points[:, :2], p)
    nonzero = mask.nonzero().numpy().flatten()
    pointgrid = test_mesh.extract_points(nonzero, adjacent_cells=False, include_cells=True)
    driving_meshes.append(pointgrid)
    driving_points.append(extract_flipper_points_from_mesh(pointgrid, 200))

2024-12-01 21:12:08.791 ( 377.911s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:08.932 ( 378.052s) [          58CB80]      vtkDelaunay3D.cxx:514   WARN| vtkDelaunay3D (0x34d435b10): 38 degenerate triangles encountered, mesh quality suspect
2024-12-01 21:12:09.165 ( 378.285s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:09.301 ( 378.421s) [          58CB80]      vtkDelaunay3D.cxx:514   WARN| vtkDelaunay3D (0x34d251330): 31 degenerate triangles encountered, mesh quality suspect
2024-12-01 21:12:09.524 ( 378.643s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:09.659 ( 378.779s) [          58CB80]      vtkDelaunay3D.cxx:514   WARN| vtkDelaunay3D (0x34d42c220): 30 degenerate triangles encountered, mesh quality suspect
2024-12-01 21:12:09.885 ( 379.004s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor

In [35]:
driving_meshes[0].plot(render_points_as_spheres=True)

Widget(value='<iframe src="http://localhost:63615/index.html?ui=P_0x310d4e850_11&reconnect=auto" class="pyvist…

In [36]:
delaunay = driving_meshes[0].delaunay_3d()
delaunay.plot()

2024-12-01 21:12:10.430 ( 379.549s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:10.564 ( 379.684s) [          58CB80]      vtkDelaunay3D.cxx:514   WARN| vtkDelaunay3D (0x3425a73b0): 38 degenerate triangles encountered, mesh quality suspect


Widget(value='<iframe src="http://localhost:63615/index.html?ui=P_0x343aaa990_12&reconnect=auto" class="pyvist…

In [37]:
surface = delaunay.extract_surface()
surface_points = pv.PolyData(surface.points)
surface_points.plot(render_points_as_spheres=True, point_size=10,)

Widget(value='<iframe src="http://localhost:63615/index.html?ui=P_0x17c789d10_13&reconnect=auto" class="pyvist…

In [38]:
import pyacvd

In [39]:
clus = pyacvd.Clustering(surface)
clus.cluster(200, maxiter=1000, iso_try=50)

array([  0,   1,   0, ..., 102,  42, 101], dtype=int32)

In [46]:
pv.PolyData(clus.cluster_centroid).plot(render_points_as_spheres=True, point_size=10,)

Widget(value='<iframe src="http://localhost:63615/index.html?ui=P_0x36fe48d90_19&reconnect=auto" class="pyvist…

In [41]:
body_mask = points_in_oriented_box(robot_points[:, :2], body_bbox)
nonzero = body_mask.nonzero().numpy().flatten()
body_pointgrid = test_mesh.extract_points(nonzero, adjacent_cells=False, include_cells=True)

In [47]:
body_pointgrid.plot()

Widget(value='<iframe src="http://localhost:63615/index.html?ui=P_0x36fec2690_20&reconnect=auto" class="pyvist…

In [48]:
body_delaunay = body_pointgrid.delaunay_3d()
body_delaunay.plot()

2024-12-01 21:12:35.722 ( 404.841s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:35.722 ( 404.841s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:35.722 ( 404.841s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:35.722 ( 404.841s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:35.722 ( 404.841s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:35.722 ( 404.842s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:35.722 ( 404.842s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:35.722 ( 404.842s) [          58CB80]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-01 21:12:35.722 ( 404.84

Widget(value='<iframe src="http://localhost:63615/index.html?ui=P_0x396ce2ad0_21&reconnect=auto" class="pyvist…

In [54]:
body_surface = body_pointgrid.extract_surface()
body_surface_points = pv.PolyData(body_surface.points)
body_surface_points.plot(render_points_as_spheres=True, point_size=10,)

Widget(value='<iframe src="http://localhost:63615/index.html?ui=P_0x39fd925d0_24&reconnect=auto" class="pyvist…

In [56]:
pv.PolyData(pv.voxelize(body_pointgrid.extract_surface(),density=0.1,check_surface=False).points).plot(render_points_as_spheres=True, point_size=10,)

Widget(value='<iframe src="http://localhost:63615/index.html?ui=P_0x3b5c78a90_25&reconnect=auto" class="pyvist…